# "[Recommender System] Session-based Recommendation with RNN"
> 2016, Session 상황에서의 RNN 기반(GRU)의 추천 시스템

- toc: true
- badges: true
- comments: true
- categories: [Paper]
- tags: [paper, recommender system, RNN, GRU]

# Session-based Recommendation with RNN

![](../images/paper/recommendation/srnn/srnn-001.png)

## Abstract

![](../images/paper/recommendation/srnn/srnn-002.png)

- RNN을 추천시스템에 적용
- Short session-based 상황에서 Matrix Factorization은 정확도가 떨어짐
- Item-to-item 추천 방식을 short session에서 활용하기도 함
- RNN-based 방식을 통해 session-based 추천시스템을 구현할 수 있음
- RNN의 ranking loss를 변경하여 Session-based recommendation task에서 좋은 성능을 얻을 수 있음

## Introduction

- 현재까지 대부분의 session-based 추천은 간단한 방법을 사용할 수 밖에 없었음
    - User profile을 사용하지 못하고 item간 유사도, co-occurrence 등을 활용
    - E-commerce 분야에서 특히 session-based 추천을 적용하기 어려웠음
- 주로 사용되는 모델은 factor model과 neighborhood method 임
    - Factor model은 user profile이 없기 때문에 session-based 추천문제를 풀 수 없음
    - Neighborhood method는 유사도와 co-occurrnece 기반이기 때문에 session-based 추천에서 적절히 사용할 수 있음

- RNN의 sequential한 특징 덕분에 session-based 추천에 활용할 수 있음
- Sparse한 sequential 데이터를 모델링하고 추천시스템에 RNN을 적용
- 새로운 ranking loss를 task에 맞게 다시 제안
- 학습할 때,
    - 사용자가 웹사이트에 들어왔을 때 하는 첫번째 클릭 = initial input
    - 이전 클릭에 따라 사용자는 다음 클릭을 연이어서 진행함 = output
- 위의 click-stream 데이터셋을 활용하는데, 데이터 사이즈와 scalability가 매우 중요

## Recommendations with RNNs

![](../images/paper/recommendation/srnn/srnn-003.png)

### Customizing the GRU Model

![](../images/paper/recommendation/srnn/srnn-004.png)

- Input: Actual state of the session
- Output: item of the thext event in the session
- State of the session의 정의
    - Item of the actual event: one-hot vector
    - Events in the session: weighted sum of these representations

## Session-Parallel Mini-batches

![](../images/paper/recommendation/srnn/srnn-005.png)

- 기존 RNN batch 방식을 적용할 수 없음
    - Session 길이가 매우 다름
    - 시간 변화에 따라 session이 어떻게 생겨나는지를 학습하는 것이 목적
- Session-parallel mini-batchs 방법을 선택
- Session은 모두 독립이며, hidden state를 reset 함

### Sampling on the output

- 매 step 마다 모든 item에 대해서 점수를 계산하는 것은 비효율적임
- 아이템 중 subset만 뽑아서 점수를 계산하는 것이 좋음
- 다음과 같이 sampling할 때는 popularity를 고려해야 함
    - 아이템이 존재하는지 몰랐기 때문에 관심 갖지 않음
    - 유명한데도 안봤다면 좋아하지 않을 가능성이 높음
- 따라서 mini-batch에서 다른 학습데이터를 negative example로 활용
- 이는, mini-batch에서 다른 학습 데이터에 아이템이 존재할 likelihood는 popularity와 비례함

### Ranking Loss

- Pointwise: 각 아이템에 점수를 매김
- Pairwise: 유저가 선택한 아이템과 그렇지 않은 아이템을 고르고, 유저가 선택한 아이템을 선택하지 않은 아이템보다 더 선호할 확률을 최대화 함
- Listwise: 모든 아이템에 점수와 랭킹을 부여하고 랭킹을 바탕으로 정답과 비교
- 그러나 논문에서는 Pairwise Loss가 가장 성능이 좋다고 함
    - **BPR**(**Bayesian Personalized Ranking**)
    - TOP1: relative rank of the relevant item

## Experiments

- Dataset
    - RSC15: e-commerce 사이트의 click-stream 데이터
    - YouTube-like OTT: 특정 시간 이상 비디오를 시청한 이벤트 데이터
- Evaluation
    - RSC15sms 매 event마다 모든 item에 대해 ranking 측정. 반면 OTT는 상위 30,000개만 ranking 측정
    - Recall과 MRR(Mean Reciprocal Rank)을 측정
- Baseline
    - POP: Poularity-based
    - S-POP: Most popular items on current session
    - Item-KNN
    - BPR-MF

![](../images/paper/recommendation/srnn/srnn-006.png)

- Best Parameters for GRU4Rec

- **AdaGrad** > **RMSProp**
- **LSTM**, **RNN** 보다 **GRU**가 더 우수
- Session이 그렇게 길지 않기 때문에 single layer GRU도 충분
- GRU의 사이즈를 키우는 것은 도움됨
- 세션의 모든 이전 이벤트를 input으로 사용하는 것은 이전 하나만 넣는 것에 비해 성능향상 없음, **GRU** 역시 long short term memory 이슈가 있기 때문

![](../images/paper/recommendation/srnn/srnn-007.png)

- Baseline method 결과

![](../images/paper/recommendation/srnn/srnn-008.png)

- GRU4Rec 결과

## Conclusions

- Session-based Recommendation with Recurrent Neural Network(GRU)를 제안
- Session-based Recommendation을 위한 다음과 같은 방법을 제안
    - Session-parallel mini-batches
    - Mini-batch based output sampling
    - Ranking Loss function
- 이 task에 대해 다른 baseline보다 성능이 우수함을 확인